### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6950,2023-11-28,Brasil Nbb,19:00,Cerrado,Mogi,1.55,2.34,151.5,1.86,1.84,-4.5,1.98,1.74,xv5Ar399,0.645161,0.427350,0.537634,0.543478,0.072512,0.0,0.0,NaN,NaN,331.598,351.058347,1.058687,290.554,135.743466,0.467188,0.000,0.000,129.36,219.45,0.0,0.0,0.0,0.0,0.287205,0.007644,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6951,2023-11-28,Brasil Nbb,19:30,União Corinthians,Paulistano,2.97,1.36,147.5,1.80,1.86,5.5,1.97,1.74,js9EsNOF,0.336700,0.735294,0.555556,0.537634,0.071994,0.0,0.0,NaN,NaN,240.494,150.480196,0.625713,140.774,27.881728,0.198060,0.000,0.000,159.84,162.74,0.0,0.0,0.0,0.0,0.525839,0.023184,0.087674,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6952,2023-11-28,Brasil Nbb,20:00,Caxias do Sul,São Paulo,2.97,1.36,153.5,1.86,1.84,5.5,2.00,1.72,Y5bJtsvM,0.336700,0.735294,0.537634,0.543478,0.071994,0.0,0.4,NaN,NaN,220.422,111.024985,0.503693,140.742,21.558049,0.153174,0.000,138.386,175.48,128.79,0.0,0.0,0.0,0.0,0.525839,0.007644,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,-3.41,-0.682,-0.527859,0.000000,0.0,0.000000
6953,2023-11-28,Eua Nba,00:30,Los Angeles Clippers,Denver Nuggets,1.25,4.70,217.5,1.98,1.97,-12.5,2.06,1.73,KUAmHSw7,0.800000,0.212766,0.505051,0.507614,0.012766,0.0,0.6,NaN,NaN,164.744,24.021037,0.145808,167.134,19.978621,0.119537,158.924,170.110,202.23,142.76,0.0,0.0,0.0,0.0,0.820006,0.003580,0.123137,-0.41,-0.082,-3.048780,0.655036,0.6,-0.055036,-5.00,-1.000,-3.700000,0.592222,0.5,-0.092222
6954,2023-11-28,Eua Nba,21:30,Boston Celtics,Chicago Bulls,1.13,7.70,215.5,1.83,2.04,-13.5,2.10,1.84,SIpRcXYn,0.884956,0.129870,0.546448,0.490196,0.014826,0.6,0.6,NaN,NaN,159.848,33.679185,0.210695,271.632,57.874313,0.213061,157.108,348.492,140.12,250.70,0.0,0.0,0.0,0.0,1.052252,0.076740,0.093324,1.35,0.270,0.481481,0.780313,0.7,-0.080313,-5.00,-1.000,-6.700000,0.405058,0.2,-0.205058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2023-11-28,México Lnbp,23:15,Astros,Libertadores,1.41,2.76,174.5,1.85,1.85,-7.5,2.09,1.63,txR8mvEo,0.709220,0.362319,0.540541,0.540541,0.071539,0.0,0.0,NaN,NaN,176.670,36.871538,0.208703,154.038,47.153335,0.306115,0.000,0.000,129.86,235.20,0.0,0.0,0.0,0.0,0.457839,0.000000,0.174876,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7039,2023-11-28,Rússia Copa Da Rússia,09:00,Enisey Krasnoyarsk F,Nadezhda F,3.00,16.50,132.5,1.83,1.85,4.5,2.04,1.68,MiY2lyhM,0.333333,0.060606,0.546448,0.540541,-0.606061,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.979071,0.007686,0.136859,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7040,2023-11-28,Rússia Copa Da Rússia,12:00,Samara F,MBA Moscow F,2.98,15.50,137.5,1.80,1.90,4.5,2.01,1.70,6i4IfctJ,0.335570,0.064516,0.555556,0.526316,-0.599913,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.958114,0.038222,0.118169,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7041,2023-11-28,Rússia Copa Da Rússia,12:00,Sparta & K Vidnoe F,Dynamo Kursk F,9.30,23.00,131.5,1.85,1.85,13.5,2.09,1.63,t6fTCvdm,0.107527,0.043478,0.540541,0.540541,-0.848995,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.599837,0.0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,21:30,Eua Nba,Boston Celtics,Chicago Bulls,215.5,1.83,0.9999,Over
5,21:30,Eua Nba,Brooklyn Nets,Toronto Raptors,220.5,1.80,1.0000,Over
6,21:30,Eua Nba,Cleveland Cavaliers,Atlanta Hawks,235.5,1.89,1.0000,Over
7,21:30,Eua Nba,Miami Heat,Milwaukee Bucks,224.5,1.88,1.0000,Over
8,21:30,Eua Nba,New York Knicks,Charlotte Hornets,221.5,1.83,0.9999,Over
9,22:00,Eua Nba,Minnesota Timberwolves,Oklahoma City Thunder,224.5,1.88,1.0000,Over
10,22:30,Eua Nba,Dallas Mavericks,Houston Rockets,231.5,1.86,1.0000,Over
